<a href="https://colab.research.google.com/github/fumio125/ou_dip/blob/master/ou_dip_08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np  # PythonのOpenCVでは、画像はnumpyのarrayとして管理される
from google.colab.patches import cv2_imshow # colab内で画像表示関数がうまく動かないので、パッチが提供されている

# Googleドライブへのマウント
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/Colab Notebooks/ou_dip/"

In [ ]:
src = cv2.imread("prims.png", cv2.IMREAD_GRAYSCALE) # 画像はgithub内に
noise = np.random.normal(0.0, 32.0, src.shape) # ノイズ付加
src = np.clip(src + noise,0,255).astype(np.uint8)
cv2_imshow(src)

In [ ]:
# 閾値128で2値化
retval, binary = cv2.threshold(src,128,255,cv2.THRESH_BINARY)
print("Binarized by", retval)
cv2_imshow(binary)

# 大津の2値化
retval, binary_otsu = cv2.threshold(src,0,255,cv2.THRESH_OTSU)
print("Otsu's thresholding: threshold=",retval)
cv2_imshow(binary_otsu)

In [ ]:
# 8近傍カーネル（8近傍と自身がすべて1）
kernel8 = np.ones((3,3),np.uint8)

# dilation
print("dilation")
dilation = cv2.dilate(binary_otsu,kernel8,iterations = 1)
cv2_imshow(dilation)

# erosion
print("erosion")
erosion = cv2.erode(binary_otsu,kernel8,iterations = 1)
cv2_imshow(erosion)

# opening
print("opening")
opening = cv2.morphologyEx(binary_otsu, cv2.MORPH_OPEN, kernel8)
cv2_imshow(opening)

# opening
print("closing")
closing = cv2.morphologyEx(binary_otsu, cv2.MORPH_CLOSE, kernel8)
cv2_imshow(closing)

# a combination of opening & closing
print("combination of opening & closing")
dst = cv2.morphologyEx(binary_otsu, cv2.MORPH_CLOSE, kernel8,iterations=2)
dst = cv2.morphologyEx(dst, cv2.MORPH_OPEN, kernel8,iterations=2)
cv2_imshow(dst)

In [ ]:
# labeling
n_labels, labels, data, center = cv2.connectedComponentsWithStats(dst)
print("[labeling] n_labels:", n_labels)

In [ ]:
# first component (0: background)
obj = ((labels==1)*255).astype(np.uint8)
cv2_imshow(obj)

# moments
M = cv2.moments(obj)
print("moments", M)

# area
print("area =",M['m00'])

# center
cx = (M['m10']/M['m00'])
cy = (M['m01']/M['m00'])
print("center = (",cx,",",cy,")")

# 主軸を求めてみよう（提出不要）



参考

https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_thresholding/py_thresholding.html
https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_morphological_ops/py_morphological_ops.html